# This project is divided into two parts:

In the first, we will be trying out scraping using APIs.

The second and more intricate part will be an in-depth scrape into e-bay's playstation 4 search pages in order to later using basic statistical analysis to examine which sellers advertise and sponsor on eBay.

## Part 1

Go to http://numbersapi.com and familiarize yourself with the API.

a) Write a program that accesses all trivia results for the numbers from 0 (zero) to 99 using batch requests only (One single query for all the numbers). Print the output result to the screen in the format of [3-digit number with leading zeros] - [TRIVIA] (One number per each line). E.g.,<br><br>
000 - 0 is the coldest possible temperature old the Kelvin scale.<br>
001 - 1 is the number of dimensions of a line.<br>
002 - 2 is the price in cents per acre the USA bought Alaska from Russia.

In [1]:
# Begin by importing all required packages
import requests
import json

In [2]:
# Create list of numbers from 000 to 099 with leading zeros
numbers = list(range(100))
numbers = [str(item) for item in numbers]
numbers_leading_zeros = [str(item).zfill(3) for item in numbers]

According to the api website, a number range (inclusive) is specified as min..max. Separate multiple ranges and individual numbers with a comma.

In [3]:
# Concatenating all numbers of the list and separating with a comma
numbers_join = ','.join(numbers)

# Now we create a URL in the same format the website has mentioned
url = "http://numbersapi.com/"+numbers_join

In [4]:
# Requesting the website's access to the server and storing the response
response = requests.get(url)
# Parsing the response
trivia = json.loads(response.text)
# Extracting values from dictionary
trivia_values = list(trivia.values())

In [5]:
# Print by concatenating numbers with leading zeros, "-" and trivia values
for n, t in zip(numbers_leading_zeros, trivia_values):
    print(n,"-", t)

000 - 0 is the coldest possible temperature old the Kelvin scale.
001 - 1 is the number of Gods in monotheism.
002 - 2 is the first magic number in physics.
003 - 3 is the number of novels or films in a trilogy and the number of interconnected works of art in a triptych.
004 - 4 is the number of completed, numbered symphonies by Johannes Brahms.
005 - 5 is the number of babies born in a quintuplet.
006 - 6 is the number of symbolic foods placed on the Passover Seder Plate.
007 - 7 is the number of days in a week.
008 - 8 is the number of legs that arachnids have.
009 - 9 is the number of circles of Hell in Dante's Divine Comedy.
010 - 10 is the highest score possible in Olympics gymnastics competitions.
011 - 11 is the miles per hours that the fastest moving land snake, the Black Mamba, can move.
012 - 12 is the number of constellations in the ecliptic (or signs of the zodiac).
013 - 13 is the number of loaves in a "baker's dozen".
014 - 14 is the number of pieces the body of Osiris wa

## Part 2 

a) Search for buy-it-now listings of "playstation 4 slim" and limit the number of items to 100 per page. 

Identify all the sponsored items.

For the first 10 pages of 100 items/page, save all the URLs of sponsored items' pages to the file "sponsored.txt"  and all the URLs of non-sponsored items' pages to the file "non-sponsored.txt" in the same directory as your code. (One URL per line in each file)

In [6]:
# Import neccessary packages
import time
import re
from bs4 import BeautifulSoup
from requests import get
import os
import os.path

In [7]:
# Adding user agent as Mozilla to make the server believe that the request came from a browser
user_agent = 'Mozilla/5.0'
# Adding the user agent to the request header
headers = {'User-Agent': user_agent}

In [8]:
# Creates the list of page numbers from 1-10 as in the url as 01, 02, etc
page_numbers = ["{0:02}".format(i+1) for i in range(10)]
page_numbers

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

In [9]:
# Loops 10 times over the page number list, one per page number
file_name = []
for i in range(10):
    # Concatenates page number with the url to create the url for each result page
    url = "https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&rt=nc&LH_BIN=1&_ipg=100&_pgn=" + page_numbers[i]
    # Requesting the ebay website's (with specific page number) access to the server and storing the response
    res = get(url, headers = headers)
    # Parsing the response and storing the resulting html
    data = BeautifulSoup(res.text, 'html.parser')
    # Creates the file name by concatenating the page numbers
    file_name.append("playstation_4_slim_" + page_numbers[i] + ".htm")
    # Writes the output to the html file
    with open(file_name[i], "w", encoding='utf-8') as file:
        file.write(str(data))
    # Waits idle for 2 secs before moving on to the next iteration of the loop
    #time.sleep(2)

In [10]:
# Loops 10 times over the files created, once for each file
# Let's create the list of all products and sponsored products
sponsored_products = []
total_products = []

for name in file_name:
    # Reads the file
    with open(name, "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        # Finds the h3 tags with particular class, relating to all items from the results shown
        total_titles = content.findAll("h3", {"class": "s-item__title"})
        # Finds the h3 tags with particular class, relating to the sponsored items from the results shown
        sponsored_titles = content.findAll("h3", {"class": "s-item__title s-item__title--has-tags"})
        # adds each title from the h3 tag to a list of total_products
        for title in total_titles:
            total_products.append(title)
        # adds each title from the h3 tag to a list of sponsored_products
        for title in sponsored_titles:
            sponsored_products.append(title)

### Sponsored Products

In [11]:
# Loops over the sponsored products list
for each in sponsored_products:
    # Prints the url of the each title by traversing to the parent of the h3 tag(which is <a> tag)
    # and getting the web link from href attribute
    url_simplify = re.sub("(.*)\\?.*", r"\1\n", each.parent.get('href'))
    # Reads file
    file = open("sponsored.txt","a")
    # Storing content into text file
    file.write(url_simplify)

### Unsponsored Products

In [12]:
for i in sponsored_products: 
    try: 
        total_products.remove(i) 
    except ValueError: 
        pass
    unsponsored_products = total_products

In [13]:
# Loops over the unsponsored products list
for each in unsponsored_products:
    # Prints the url of the each title by traversing to the parent of the h3 tag(which is <a> tag)
    # and getting the web link from href attribute
    url_simplify = re.sub("(.*)\\?.*", r"\1\n", each.parent.get('href'))
    # Reads file
    file = open("non-sponsored.txt","a")
    # Storing content into text file
    file.write(url_simplify)

b) Create two folders in the same directory as your code and name them "sponsored" and "non-sponsored". Write a program that opens the two files in (a) and downloads each of the pages (URLs) into the folders "sponsored" and "non-sponsored". Each file should be named as "< item-id>.htm" where you replace "item-id" with the ID of the item you are saving.

In [14]:
# Create sponsored folder
if not os.path.exists("sponsored"):
    os.makedirs("sponsored")
# Create non-sponsored folder
if not os.path.exists("non-sponsored"):
    os.makedirs("non-sponsored")

In [15]:
# Creating lists of the sponsored and non-sponsored URLs 
sponsored_url = open("sponsored.txt").read().splitlines()
non_sponsored_url = open("non-sponsored.txt").read().splitlines()

In [16]:
# Extracting ids from sponsored url list
ids_sponsored = [x[-12:] for x in sponsored_url]
# Extracting ids from non-sponsored url list
ids_non_sponsored = [x[-12:] for x in non_sponsored_url]

In [17]:
# Defining the working directory that files will be saved to
sponsored_working_directory = os.getcwd()+"/sponsored"
non_sponsored_working_directory = os.getcwd()+"/non-sponsored"

### Downloading Sponsored Pages to Sponsored Folder

In [18]:
# Loops over all URLs in the sponsored url list
sponsored_file_name = []
# Check number of urls in the list for looping
length_sponsored = len(sponsored_url)
for url,i in zip(sponsored_url,range(length_sponsored)):
    #Requesting the ebay's website (for sponsored listings) access to the server and storing the response
    res = get(url, headers = headers)
    # Parsing the response and storing the resulting html
    data = BeautifulSoup(res.text, 'html.parser')
    # Creates file names by indexing the ids from ids_sponsored list
    sponsored_file_name.append(ids_sponsored[i] + ".htm")
    # Writes output to html file in the correct working directory
    with open(os.path.join(sponsored_working_directory,sponsored_file_name[i]), "w", encoding='utf-8') as file:
        file.write(str(data))
    # Waits idle for 2 secs before moving on to the next iteration of the loop
    time.sleep(2)

### Downloading Non-Sponsored Pages to Non-Sponsored Folder

In [20]:
# Loops over all URLs in the non-sponsored url list
non_sponsored_file_name = []
# Check number of urls in the list for looping
length_non_sponsored = len(non_sponsored_url)
for url,i in zip(non_sponsored_url,range(length_non_sponsored)):
    #Requesting the ebay's website (for non-sponsored listings) access to the server and storing the response
    res = get(url, headers = headers)
    # Parsing the response and storing the resulting html
    data = BeautifulSoup(res.text, 'html.parser')
    # Creates file names by indexing the ids from ids_non_sponsored list
    non_sponsored_file_name.append(ids_non_sponsored[i] + ".htm")
    # Writes output to html file in the correct working directory
    with open(os.path.join(non_sponsored_working_directory,non_sponsored_file_name[i]), "w", encoding='utf-8') as file:
        file.write(str(data))
    # Waits idle for 2 secs before moving on to the next iteration of the loop
    time.sleep(2)

c) Write a separate piece of code that loops through the pages you downloaded in (b) and opens and parses them into a Python or Java xxxxsoup-object. Identify and select:

seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).

In [22]:
# sponsored seller_name
sponsored_seller_name = []

for name in sponsored_file_name:
    # Reads the file
    with open(os.path.join(sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Main seller information
        # Finds the div tags with class "rsp-c", relating to the seller information items from the results shown
        seller_main = content.findAll("div", attrs={'class':"rsp-c"})

        # seller_name
        # Using a for loop, look for all span nests with class "mbg-nw" and extract the content within the nest
        for i in range(len(seller_main)):
            seller = seller_main[i].findAll("span", class_ = "mbg-nw")[i].text
            # Adding each seller to a final list: seller_name
            sponsored_seller_name.append(seller)

In [144]:
#sponsored_seller_name
len(sponsored_seller_name)

126

In [196]:
# non-sponsored seller_name
non_sponsored_seller_name = []

for name in non_sponsored_file_name:
    # Reads the file
    with open(os.path.join(non_sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Main seller information
        # Finds the span tags with class "mbg-nw", relating to the seller information items from the results shown
        seller_main = content.findAll("span", class_ = "mbg-nw")
        seller_main_str = str(seller_main)
        
        # Adding each seller to a final list: seller_name
        non_sponsored_seller_name.append(seller_main_str)

In [238]:
non_sponsored_seller_name_final = []
for i in range(len(non_sponsored_seller_name)):
    c1 = re.sub('\[<span class="mbg-nw">','', non_sponsored_seller_name[i])
    c2 = re.sub('\</span>]','', c1)
    c3 = re.sub('\</span>','',c2)
    c4 = re.sub('\s\<span class="mbg-nw">','',c3)
    #c3 = re.sub('\[<div class="si-sp-fb" id="si-fb">','', c2)
    #c4 = re.sub('\\xa0Positive feedback</div>]','', c3)
    #c5 = re.sub('\[]','', c4)
    non_sponsored_seller_name_final.append(c4)

In [27]:
# sponsored seller_score
sponsored_seller_score = []

for name in sponsored_file_name:
    # Reads the file
    with open(os.path.join(sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Main seller information
        # Finds the div tags with id "si-fb", relating to the seller score information from the results shown
        score = content.findAll("div", attrs={'id':"si-fb"})
        score_str = str(score)
        
        # Adding each seller to a final list: seller_name
        sponsored_seller_score.append(score_str)

In [28]:
# Now time to take off everything but the numbers!
# It's long but you can't say i'm not thinking outside the box!
sponsored_seller_score_final = []
for i in range(len(sponsored_seller_score)):
    c1 = re.sub('\[<div id="si-fb">','', sponsored_seller_score[i])
    c2 = re.sub('\[<div class="si-mbg" id="si-fb">','', c1)
    c3 = re.sub('\[<div class="si-sp-fb" id="si-fb">','', c2)
    c4 = re.sub('\\xa0Positive feedback</div>]','', c3)
    c5 = re.sub('\[]','', c4)
    sponsored_seller_score_final.append(c5)

In [30]:
# non-sponsored seller_score
non_sponsored_seller_score = []

for name in non_sponsored_file_name:
    # Reads the file
    with open(os.path.join(non_sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Main seller information
        # Finds the div tags with id "si-fb", relating to the seller score information from the results shown
        score = content.findAll("div", attrs={'id':"si-fb"})
        score_str = str(score)
        #print(score)
        
        # Adding each seller to a final list: seller_name
        non_sponsored_seller_score.append(score_str)

In [31]:
# Now time to take off everything but the numbers!
# It's long but you can't say i'm not thinking outside the box!
non_sponsored_seller_score_final = []
for i in range(len(non_sponsored_seller_score)):
    c1 = re.sub('\[<div id="si-fb">','', non_sponsored_seller_score[i])
    c2 = re.sub('\[<div class="si-mbg" id="si-fb">','', c1)
    c3 = re.sub('\[<div class="si-sp-fb" id="si-fb">','', c2)
    c4 = re.sub('\\xa0Positive feedback</div>]','', c3)
    c5 = re.sub('\[]','', c4)
    non_sponsored_seller_score_final.append(c5)

In [33]:
# sponsored item price
sponsored_item_price = []

for name in sponsored_file_name:
    # Reads the file
    with open(os.path.join(sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Main price nest
        # Finds the span tags with class "notranslate" and id "prcIsum", relating to the seller information items from the results shown
        prices = content.findAll("span", attrs={'class':"notranslate", 'id':"prcIsum"})
        prices_str = str(prices)
        
        # Adding each price to a final list
        sponsored_item_price.append(prices_str)

In [34]:
# Now time to take off everything but the numbers!
# It's long but you can't say i'm not thinking outside the box!
sponsored_item_price_final = []
for i in range(len(sponsored_item_price)):
    c1 = re.sub('\[<span class="notranslate" content="[0-9]{3}.[0-9]{2}" id="prcIsum" itemprop="price" style="">','', sponsored_item_price[i])
    c2 = re.sub('\[<span class="notranslate" content="[0-9]{3}.[0-9]{1}" id="prcIsum" itemprop="price" style="">','', c1)
    c3 = re.sub('/ea','',c2)
    c4 = re.sub('\</span>]','',c3)
    c5 = re.sub('\[]','',c4)
    sponsored_item_price_final.append(c5)

In [36]:
#Now let's split price and currencies!
sponsored_item_price_final_currency = []
for i in range(len(sponsored_item_price_final)):
    c1 = re.sub('\$.*',"$",sponsored_item_price_final[i])
    c2 = re.sub('GBP.*',"GBP",c1)
    sponsored_item_price_final_currency.append(c2)

In [38]:
#Now let's take the price values!
sponsored_item_price_final_value = []
for i in range(len(sponsored_item_price_final)):
    c1 = re.sub('US\s\$',"",sponsored_item_price_final[i])
    c2 = re.sub('GBP\s','',c1)
    sponsored_item_price_final_value.append(c2)

In [40]:
# non-sponsored item price
non_sponsored_item_price = []

for name in non_sponsored_file_name:
    # Reads the file
    with open(os.path.join(non_sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Main price nest
        # Finds the span tags with class "notranslate" and id "prcIsum", relating to the seller information items from the results shown
        prices = content.findAll("span", attrs={'class':"notranslate", 'id':"prcIsum"})
        prices_str = str(prices)
        
        # Adding each price to a final list
        non_sponsored_item_price.append(prices_str)

In [41]:
# Now time to take off everything but the numbers!
# It's long but you can't say i'm not thinking outside the box!
non_sponsored_item_price_final = []
for i in range(len(non_sponsored_item_price)):
    c1 = re.sub('\[<span class="notranslate" content="[0-9]{3}.[0-9]{2}" id="prcIsum" itemprop="price" style="">','', non_sponsored_item_price[i])
    c2 = re.sub('\[<span class="notranslate" content="[0-9]{3}.[0-9]{1}" id="prcIsum" itemprop="price" style="">','', c1)
    c3 = re.sub('/ea','',c2)
    c4 = re.sub('\</span>]','',c3)
    c5 = re.sub('\[]','',c4)
    c6 = re.sub('\[<span class="notranslate" content="[0-9]{2}.[0-9]{1}" id="prcIsum" itemprop="price" style="">','', c5)
    c7 = re.sub('\[<span class="notranslate" content="[0-9]{4}.[0-9]{2}" id="prcIsum" itemprop="price" style="">','', c6)
    non_sponsored_item_price_final.append(c7)

In [277]:
#Now let's split price and currencies!
non_sponsored_item_price_final_currency = []
for i in range(len(non_sponsored_item_price_final)):
    c1 = re.sub('\$.*',"$",non_sponsored_item_price_final[i])
    c2 = re.sub('GBP.*',"GBP",c1)
    c3 = re.sub('\[<span class="notranslate" id="prcIsum" style="">\n\t\t\t\t\t\t\t\t\t\t',"",c2)
    non_sponsored_item_price_final_currency.append(c3)

In [45]:
#Now let's take the price values!
non_sponsored_item_price_final_value = []
for i in range(len(non_sponsored_item_price_final)):
    c1 = re.sub('US\s\$',"",non_sponsored_item_price_final[i])
    c2 = re.sub('GBP\s','',c1)
    c3 = re.sub('AU\s\$',"",c2)
    non_sponsored_item_price_final_value.append(c3)

In [47]:
# sponsored items sold
sponsored_items_sold = []

for name in sponsored_file_name:
    # Reads the file
    with open(os.path.join(sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Main price nest
        # Finds the a tags with class "vi-txt-underline", relating to the seller information items from the results shown
        items_sold = content.findAll("a", attrs={'class':"vi-txt-underline"})
        items_sold_str = str(items_sold)
        
        # Adding each item sold to a final list
        sponsored_items_sold.append(items_sold_str)

In [48]:
# Now time to take off everything but the numbers!
sponsored_items_sold_final = []
for i in range(len(sponsored_items_sold)):
    c1 = re.sub('\[<a.*">','', sponsored_items_sold[i])
    c2 = re.sub('\s.*','',c1)
    c3 = re.sub('\[]','',c2)
    sponsored_items_sold_final.append(c3)

In [50]:
# non-sponsored items sold
non_sponsored_items_sold = []

for name in non_sponsored_file_name:
    # Reads the file
    with open(os.path.join(non_sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Main price nest
        # Finds the a tags with class "vi-txt-underline", relating to the seller information items from the results shown
        items_sold = content.findAll("a", attrs={'class':"vi-txt-underline"})
        items_sold_str = str(items_sold)
        
        # Adding each item sold to a final list
        non_sponsored_items_sold.append(items_sold_str)

In [51]:
# Now time to take off everything but the numbers!
non_sponsored_items_sold_final = []
for i in range(len(non_sponsored_items_sold)):
    c1 = re.sub('\[<a.*">','', non_sponsored_items_sold[i])
    c2 = re.sub('\s.*','',c1)
    c3 = re.sub('\[]','',c2)
    non_sponsored_items_sold_final.append(c3)

In [53]:
# sponsored best offer available
sponsored_best_offer = []

for name in sponsored_file_name:
    # Reads the file
    with open(os.path.join(sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Main price nest
        # Finds the span tags with id "boBtn_btn_parent", relating to the seller information items from the results shown
        best_offer = content.findAll("span", attrs={'id':"boBtn_btn_parent"})
        best_offer_str = str(best_offer)
        
        # Adding each item sold to a final list
        sponsored_best_offer.append(best_offer_str)

In [54]:
# Now categorize available/non-available offers!
sponsored_best_offer_final = []
for i in range(len(sponsored_best_offer)):
    c1 = re.sub('\[<+',"available",sponsored_best_offer[i])
    c2 = re.sub('available(.|\s)*\S(.|\s)*',"available",c1)
    c3 = re.sub('\[]','unavailable',c2)
    sponsored_best_offer_final.append(c3)

In [56]:
# non-sponsored best offer available
non_sponsored_best_offer = []

for name in non_sponsored_file_name:
    # Reads the file
    with open(os.path.join(non_sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Main price nest
        # Finds the span tags with id "boBtn_btn_parent", relating to the seller information items from the results shown
        best_offer = content.findAll("span", attrs={'id':"boBtn_btn_parent"})
        best_offer_str = str(best_offer)
        
        # Adding each item sold to a final list
        non_sponsored_best_offer.append(best_offer_str)

In [57]:
# Now categorize available/non-available offers!
non_sponsored_best_offer_final = []
for i in range(len(non_sponsored_best_offer)):
    c1 = re.sub('\[<+',"available",non_sponsored_best_offer[i])
    c2 = re.sub('available(.|\s)*\S(.|\s)*',"available",c1)
    c3 = re.sub('\[]','unavailable',c2)
    non_sponsored_best_offer_final.append(c3)

In [59]:
# Sponsored Title
sponsored_title = []

for name in sponsored_file_name:
    # Reads the file
    with open(os.path.join(sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Finds the h1 tags with class "it-ttl", relating to the seller information items from the results shown
        title = content.findAll("h1", attrs={'class':"it-ttl"})
        title_str = str(title)
        
        # Adding each item sold to a final list
        sponsored_title.append(title_str)

In [60]:
# Now pulling out the titles!
sponsored_title_final = []
for i in range(len(sponsored_title)):
    c1 = re.sub('\[<h1 class="it-ttl" id="itemTitle" itemprop="name"><span class="g-hdn">Details about  \xa0</span>',"",sponsored_title[i])
    c2 = re.sub('\</h1>]',"",c1)
    sponsored_title_final.append(c2)

In [62]:
# non-Sponsored Title
non_sponsored_title = []

for name in non_sponsored_file_name:
    # Reads the file
    with open(os.path.join(non_sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Finds the h1 tags with class "it-ttl", relating to the seller information items from the results shown
        title = content.findAll("h1", attrs={'class':"it-ttl"})
        title_str = str(title)
        
        # Adding each item sold to a final list
        non_sponsored_title.append(title_str)

In [63]:
# Now pulling out the titles!
non_sponsored_title_final = []
for i in range(len(non_sponsored_title)):
    c1 = re.sub('\[<h1 class="it-ttl" id="itemTitle" itemprop="name"><span class="g-hdn">Details about  \xa0</span>',"",non_sponsored_title[i])
    c2 = re.sub('\</h1>]',"",c1)
    non_sponsored_title_final.append(c2)

In [65]:
# Sponsored returns allowed
sponsored_returns = []

for name in sponsored_file_name:
    # Reads the file
    with open(os.path.join(sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Finds the span tags with id "vi-ret-accrd-txt", relating to the seller information items from the results shown
        returns = content.findAll("span", attrs={'id':"vi-ret-accrd-txt"})
        returns_str = str(returns)
        
        # Adding each item sold to a final list
        sponsored_returns.append(returns_str)

In [66]:
# Now pulling out the return details!
sponsored_returns_final = []
for i in range(len(sponsored_returns)):
    c1 = re.sub('\[<span id="vi-ret-accrd-txt">',"",sponsored_returns[i])
    c2 = re.sub('\</span>]',"",c1)
    c3 = re.sub('\[<span class="vi-no-ret-accrd-txt" id="vi-ret-accrd-txt">','',c2)
    c4 = re.sub('\\xa0'," ",c3)
    sponsored_returns_final.append(c4)

In [68]:
# non-Sponsored returns allowed
non_sponsored_returns = []

for name in non_sponsored_file_name:
    # Reads the file
    with open(os.path.join(non_sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Finds the span tags with id "vi-ret-accrd-txt", relating to the seller information items from the results shown
        returns = content.findAll("span", attrs={'id':"vi-ret-accrd-txt"})
        returns_str = str(returns)
        
        # Adding each item sold to a final list
        non_sponsored_returns.append(returns_str)

In [69]:
# Now pulling out the return details!
non_sponsored_returns_final = []
for i in range(len(non_sponsored_returns)):
    c1 = re.sub('\[<span id="vi-ret-accrd-txt">',"",non_sponsored_returns[i])
    c2 = re.sub('\</span>]',"",c1)
    c3 = re.sub('\[<span class="vi-no-ret-accrd-txt" id="vi-ret-accrd-txt">','',c2)
    c4 = re.sub('\\xa0'," ",c3)
    non_sponsored_returns_final.append(c4)

In [71]:
# Sponsored Shipping Price
sponsored_shipping_price = []

for name in sponsored_file_name:
    # Reads the file
    with open(os.path.join(sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Finds the span tags with id "fshippingCost", relating to the seller information items from the results shown
        shipping = content.findAll("span", attrs={'id':"fshippingCost"})
        shipping_str = str(shipping)
        
        # Adding each item sold to a final list
        sponsored_shipping_price.append(shipping_str)

In [72]:
# Now pulling out the shipping prices!
sponsored_shipping_price_final = []
for i in range(len(sponsored_shipping_price)):
    c1 = re.sub('\[<span class="notranslate sh-cst" id="fshippingCost">\n<span>',"",sponsored_shipping_price[i])
    c2 = re.sub('\</span>\n</span>]',"",c1)
    c3 = re.sub('\[<span class="notranslate sh-fr-cst" id="fshippingCost">\n<span>FREE',"[]",c2)
    c4 = re.sub('</span>\n<span class(.|\s)*\S(.|\s)*',"",c3)
    c5 = re.sub('\[]','',c4)
    sponsored_shipping_price_final.append(c5)

In [74]:
#Now let's split price and currencies!
sponsored_shipping_price_final_currency = []
for i in range(len(sponsored_shipping_price_final)):
    c1 = re.sub('\$.*',"$",sponsored_shipping_price_final[i])
    c2 = re.sub('GBP.*',"GBP",c1)
    sponsored_shipping_price_final_currency.append(c2)

In [76]:
#Now let's take the price values!
sponsored_shipping_price_final_value = []
for i in range(len(sponsored_shipping_price_final)):
    c1 = re.sub('\$',"",sponsored_shipping_price_final[i])
    c2 = re.sub('AU',"",c1)
    c3 = re.sub('C ',"",c2)
    c4 = re.sub('GBP ',"",c3)
    sponsored_shipping_price_final_value.append(c4)

In [78]:
# non- Sponsored Shipping Price
non_sponsored_shipping_price = []

for name in non_sponsored_file_name:
    # Reads the file
    with open(os.path.join(non_sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Finds the span tags with id "fshippingCost", relating to the seller information items from the results shown
        shipping = content.findAll("span", attrs={'id':"fshippingCost"})
        shipping_str = str(shipping)
        
        # Adding each item sold to a final list
        non_sponsored_shipping_price.append(shipping_str)

In [79]:
# Now pulling out the shipping prices!
non_sponsored_shipping_price_final = []
for i in range(len(non_sponsored_shipping_price)):
    c1 = re.sub('\[<span class="notranslate sh-cst" id="fshippingCost">\n<span>',"",non_sponsored_shipping_price[i])
    c2 = re.sub('\</span>\n</span>]',"",c1)
    c3 = re.sub('\[<span class="notranslate sh-fr-cst" id="fshippingCost">\n<span>FREE',"[]",c2)
    c4 = re.sub('</span>\n<span class(.|\s)*\S(.|\s)*',"",c3)
    c5 = re.sub('\[]','',c4)
    non_sponsored_shipping_price_final.append(c5)

In [81]:
#Now let's split price and currencies!
non_sponsored_shipping_price_final_currency = []
for i in range(len(non_sponsored_shipping_price_final)):
    c1 = re.sub('\$.*',"$",non_sponsored_shipping_price_final[i])
    non_sponsored_shipping_price_final_currency.append(c1)

In [83]:
#Now let's take the price values!
non_sponsored_shipping_price_final_value = []
for i in range(len(non_sponsored_shipping_price_final)):
    c1 = re.sub('\$',"",non_sponsored_shipping_price_final[i])
    c2 = re.sub('AU',"",c1)
    non_sponsored_shipping_price_final_value.append(c2)

In [85]:
# Sponsored Condition
sponsored_condition = []

for name in sponsored_file_name:
    # Reads the file
    with open(os.path.join(sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Finds the div tags with class "u-flL condText", relating to the seller information items from the results shown
        condition = content.findAll("div", attrs={'class':"u-flL condText"})
        condition_str = str(condition)
        
        # Adding each item sold to a final list
        sponsored_condition.append(condition_str)

In [86]:
# Now pulling out the conditions!
sponsored_condition_final = []
for i in range(len(sponsored_condition)):
    c1 = re.sub('\[<div class="u-flL condText" id="vi-itm-cond" itemprop="itemCondition">',"",sponsored_condition[i])
    c2 = re.sub('\</div>]',"",c1)
    #c3 = re.sub('\[<span class="notranslate sh-fr-cst" id="fshippingCost">\n<span>FREE',"[]",c2)
    #c4 = re.sub('</span>\n<span class(.|\s)*\S(.|\s)*',"",c3)
    #c5 = re.sub('\[]','',c4)
    sponsored_condition_final.append(c2)

In [88]:
# non-Sponsored Condition
non_sponsored_condition = []

for name in non_sponsored_file_name:
    # Reads the file
    with open(os.path.join(non_sponsored_working_directory,name), "r", encoding='utf-8') as file:
        # Parses the file contents and stores the resulting html
        content = BeautifulSoup(file.read(), 'html.parser')
        
        # Finds the div tags with class "u-flL condText", relating to the seller information items from the results shown
        condition = content.findAll("div", attrs={'class':"u-flL condText"})
        condition_str = str(condition)
        
        # Adding each item sold to a final list
        non_sponsored_condition.append(condition_str)

In [89]:
# Now pulling out the conditions!
non_sponsored_condition_final = []
for i in range(len(non_sponsored_condition)):
    c1 = re.sub('\[<div class="u-flL condText" id="vi-itm-cond" itemprop="itemCondition">',"",non_sponsored_condition[i])
    c2 = re.sub('\</div>]',"",c1)
    non_sponsored_condition_final.append(c2)

d) Use your code script to connect to SQL. This table should contain both sponsored and non-sponsored information and have a column that specifies which item is sponsored/non-sponsored.

In [280]:
# First, import the required package
import mysql.connector

In [281]:
# Connects to local SQL instance
db = mysql.connector.connect(host="localhost", 
                               user="root",
                               passwd="", 
                               use_pure=True)
mysql_db = db.cursor()

In [282]:
# Create table and make sure it's the one in use
mysql_db.execute("CREATE DATABASE IF NOT EXISTS eBay")
mysql_db.execute("USE eBay")

In [283]:
# Creating columns for sponsored items

seller_name1 = []
seller_score1 = []
item_price_currency1 = []
item_price_value1 = []
items_sold1 = []
best_offer1 = []
title1 = []
returns1 = []
shipping_price_currency1 = []
shipping_price_value1 = []
condition1 = []
spons_type1 = ["Sponsored"]*len(sponsored_file_name)

for i in range(len(sponsored_file_name)):
    seller_name1.append(sponsored_seller_name[i])
    seller_score1.append(sponsored_seller_score_final[i])
    item_price_currency1.append(sponsored_item_price_final_currency[i])
    item_price_value1.append(sponsored_item_price_final_value[i])
    items_sold1.append(sponsored_items_sold_final[i])
    best_offer1.append(sponsored_best_offer_final[i])
    title1.append(sponsored_title_final[i]) 
    returns1.append(sponsored_returns_final[i])
    shipping_price_currency1.append(sponsored_shipping_price_final_currency[i])
    shipping_price_value1.append(sponsored_shipping_price_final_value[i])
    condition1.append(sponsored_condition_final[i])
  

In [284]:
# Creating columns for non-sponsored items

seller_name2 = []
seller_score2 = []
item_price_currency2 = []
item_price_value2 = []
items_sold2 = []
best_offer2 = []
title2 = []
returns2 = []
shipping_price_currency2 = []
shipping_price_value2 = []
condition2 = []
spons_type2 = ["Non-Sponsored"]*len(non_sponsored_file_name)


for i in range(len(non_sponsored_file_name)):
    seller_name2.append(non_sponsored_seller_name_final[i])
    seller_score2.append(non_sponsored_seller_score_final[i])
    item_price_currency2.append(non_sponsored_item_price_final_currency[i])
    item_price_value2.append(non_sponsored_item_price_final_value[i])
    items_sold2.append(non_sponsored_items_sold_final[i])
    best_offer2.append(non_sponsored_best_offer_final[i])
    title2.append(non_sponsored_title_final[i])
    returns2.append(non_sponsored_returns_final[i])
    shipping_price_currency2.append(non_sponsored_shipping_price_final_currency[i])
    shipping_price_value2.append(non_sponsored_shipping_price_final_value[i])
    condition2.append(non_sponsored_condition_final[i])

In [285]:
# unioning columns
seller_name = seller_name1 + seller_name2
seller_score = seller_score1 + seller_score2
item_price_currency = item_price_currency1 + item_price_currency2
item_price_value = item_price_value1 + item_price_value2
items_sold = items_sold1 + items_sold2
best_offer = best_offer1 + best_offer2
title = title1 + title2
returns = returns1 + returns2
shipping_price_currency = shipping_price_currency1 + shipping_price_currency2
shipping_price_value = shipping_price_value1 + shipping_price_value2
item_condition = condition1 + condition2
spons_type = spons_type1 + spons_type2

In [286]:
# Function to convert list to floats while considering empty strings
def try_float(a):
   try:
       return float(a)
   except Exception:
       return ""

In [287]:
# Convert item_price_value and shipping_price_value
item_price_value = [try_float(item) for item in item_price_value]
shipping_price_value = [try_float(item) for item in shipping_price_value]

In [288]:
# Now time to convert to dollar_cent format
item_price_value = [x * 100 for x in item_price_value]
shipping_price_value = [x * 100 for x in shipping_price_value]

In [289]:
# Function to round list of floats while considering empty strings
def try_round(a):
   try:
       return round(a)
   except Exception:
       return None

In [290]:
# Round prices
item_price_value = [try_round(item) for item in item_price_value]
shipping_price_value = [try_round(item) for item in shipping_price_value]

In [291]:
# Changes empty strings in seller_score to None
for i, item in enumerate(seller_score[1:]):
    if not item:
        seller_score[i + 1] = None

In [292]:
# Changes empty strings in item_price_currency to None
for i, item in enumerate(item_price_currency[1:]):
    if not item:
        item_price_currency[i + 1] = None

In [293]:
# Changes empty strings in items_sold to None
for i, item in enumerate(items_sold[1:]):
    if not item:
        items_sold[i + 1] = None

In [294]:
# Changes empty strings in best_offer to None
for i, item in enumerate(best_offer[1:]):
    if not item:
        best_offer[i + 1] = None

In [295]:
# Changes empty strings in title to None
for i, item in enumerate(title[1:]):
    if not item:
        title[i + 1] = None

In [296]:
# Changes empty strings in returns to None
for i, item in enumerate(returns[1:]):
    if not item:
        returns[i + 1] = None

In [297]:
# Changes empty strings in shipping_price_currency to None
for i, item in enumerate(shipping_price_currency[1:]):
    if not item:
        shipping_price_currency[i + 1] = None

In [298]:
# Changes empty strings in item_condition to None
for i, item in enumerate(item_condition[1:]):
    if not item:
        item_condition[i + 1] = None

In [338]:
# Create table and make sure it's the one in use
mysql_db.execute("CREATE DATABASE IF NOT EXISTS eBay")
mysql_db.execute("USE eBay")

In [339]:
# Create table with variables name
mysql_db.execute("CREATE TABLE eBay_items (seller_name VARCHAR(255),seller_score VARCHAR(10),item_price_currency VARCHAR(15), item_price_value INT(20),items_sold VARCHAR(15),best_offer VARCHAR(30),title VARCHAR(255),returns VARCHAR(255),shipping_price_currency VARCHAR(20),shipping_price_value INT(20), item_condition VARCHAR(255), spons_type VARCHAR(25))")


In [340]:
# Now we need to prepare rows for the table by appending values obtained before
values = []
for i in range(len(seller_name)):
    values.append((seller_name[i], seller_score[i], item_price_currency[i], 
                   item_price_value[i], items_sold[i], 
                   best_offer[i], title[i], returns[i],
                  shipping_price_currency[i], shipping_price_value[i],
                   item_condition[i], spons_type[i]))

In [341]:
# Final Step: Store values into eBay_items table!
mysql_db.executemany("INSERT INTO eBay_items VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", values)
db.commit()

e) Use your code script to run summary stats on each item. 

In [303]:
# Import pandas! :) 
import pandas as pd

In [304]:
data = [seller_name,seller_score,item_price_currency,item_price_value,items_sold,best_offer,title,returns,shipping_price_currency,shipping_price_value,item_condition,spons_type]

In [305]:
# Create dataframe
data = {'Seller Name': seller_name,
       'Seller Score': seller_score,
       'Item Currency': item_price_currency,
       'Item Price': item_price_value,
       'Items Sold': items_sold,
       'Best Offer': best_offer,
       'Title': title,
       'Returns': returns,
       'Shipping Currency': shipping_price_currency,
       'Shipping Price': shipping_price_value,
       'Item Condition': item_condition,
       'Type of Ad': spons_type}


df = pd.DataFrame(data)

In [306]:
# Pick up first 5 rows
df.head()

,Seller Name,Seller Score,Item Currency,Item Price,Items Sold,Best Offer,Title,Returns,Shipping Currency,Shipping Price,Item Condition,Type of Ad
0,orkmastermcgee,98.1%,US $,21999.0,"1,739",unavailable,Sony PlayStation 4 (PS4) Slim 1tb Jet Black Co...,60 day returns. Buyer pays for return shipping,,NaN,Seller refurbished,Sponsored
1,newmiker,99.3%,US $,27400.0,317,unavailable,New Sony PlayStation 4 PS4 Slim 1TB Console 3...,Free 30 day returns,None,NaN,New,Sponsored
2,atechoutlet,99.1%,US $,18999.0,9,unavailable,Sony PlayStation 4 PS4 Slim 1TB Jet Black Cons...,Free 60 day returns,None,NaN,Used,Sponsored
3,leevax_llc,100%,US $,27400.0,80,unavailable,Sony PlayStation 4 PS4 Slim 1TB Console 3 Game...,Free 30 day returns,None,NaN,New,Sponsored
4,jcameras11,99.3%,US $,18995.0,None,available,Playstation 4 Console 1TB PS4 Slim CUH-2215B -...,Free 30 day returns,None,NaN,Open box,Sponsored


In [307]:
# Check stats
df.describe()

# We notice there are only 2 columns. We will need to fix the data types for the rest

,Item Price,Shipping Price
count,1108.000000,152.000000
mean,26829.938628,2044.644737
std,8014.230896,1442.341430
min,10000.000000,199.000000
25%,21999.000000,1000.000000
50%,24999.000000,1749.500000
75%,29995.500000,2500.000000
max,69999.000000,7000.000000


In [308]:
df.columns

Index(['Seller Name', 'Seller Score', 'Item Currency', 'Item Price',
       'Items Sold', 'Best Offer', 'Title', 'Returns', 'Shipping Currency',
       'Shipping Price', 'Item Condition', 'Type of Ad'],
      dtype='object')

<b>Seller Name</b>

Seller Name is non-binary/non-numerical; hence, we will get the count for each category:

In [309]:
print(df['Seller Name'].value_counts())

Pawn America,epawnamerica                    159
nationwidedistributors                        13
eflipstuff                                    10
original4u-store                               9
productdeliveries                              9
                                            ... 
esolutions_canada,esolutions_canada            1
midatlanticpawnsouth,midatlanticpawnsouth      1
lucor-8074                                     1
aniurb0                                        1
clairejyjy                                     1
Name: Seller Name, Length: 779, dtype: int64


<b>Seller Score</b>

This is a numerical variable; however, we need to adjust it

In [310]:
# Check datatype 
type(df['Seller Score'][0])

str

In [311]:
# Remove %
df['Seller Score'] = df['Seller Score'].str.replace('%','')

In [312]:
# Convert to float
df['Seller Score'] = df['Seller Score'].astype(float)

In [313]:
# Check datatype again
type(df['Seller Score'][0])

numpy.float64

<b>Item Currency</b>

Item Currency is non-binary/non-numerical; hence, we will get the count of each category level:

In [314]:
print(df['Item Currency'].value_counts())

US $    1112
Name: Item Currency, dtype: int64


<b>Item Price</b>

This is ready for final summaries (to be done later with other variables)

<b>Items Sold</b>

This datatype will need to be adjusted

In [315]:
# Check datatype 
type(df['Items Sold'][0])

str

In [316]:
# Convert to integer
# First remove commas
df['Items Sold'] = df['Items Sold'].str.replace(',', '')

# Float time
df['Items Sold'] = df['Items Sold'].astype(float)

In [317]:
# Check datatype again
type(df['Items Sold'][0])

numpy.float64

<b>Best Offer</b>

This datatype will need to be adjusted to binary showing 1 for best offer available and 0 for not available

In [318]:
# Checking unique values. Hence, we can apply binary factoring
df['Best Offer'].unique()

array(['unavailable', 'available'], dtype=object)

In [319]:
# Changing available to 1 and unavailabe to 0
df['Best Offer'] = df['Best Offer'].map({'available': 1, 'unavailable': 0})

In [320]:
# Done. We can provided in the summaries later
df['Best Offer'].head()

0    0
1    0
2    0
3    0
4    1
Name: Best Offer, dtype: int64

<b>Title</b>

Title is non-binary/non-numerical; hence, we will get the count of each category level:

In [321]:
print(df['Title'].value_counts())

Sony PlayStation 4 Slim 1TB Console - Jet Black                                     65
Sony PlayStation 4 Slim 1TB Black Console                                           51
Sony PlayStation 4 Slim 1TB Jet Black Console                                        9
Sony PlayStation 4 500GB Slim Console                                                8
Sony PlayStation 4 Slim 1TB Console - Jet Black (3002189)                            7
                                                                                    ..
Playstation 4 500GB Slim                                                             1
Sony PlayStation 4 Slim Call of Duty: WWII Limited Edition 1TB Green Camouflage…     1
Sony CUH2115B PlayStation 4 Slim 1TB Console - 5337                                  1
PlayStation 4 Slim 1TB Console Marvel's Spider-Man Bundle Very Good 7Z               1
Brand! New Sony Playstation 4 PS4 Slim bundle 1TB Console Jet Black Plus 3 Games     1
Name: Title, Length: 870, dtype: int64


<b>Returns</b>

Returns is non-binary/non-numerical; hence, we will get the count of each category level:

In [322]:
print(df['Returns'].value_counts())

Seller does not accept returns                    558
Free 30 day returns                               315
30 day returns. Buyer pays for return shipping    193
14 day returns. Buyer pays for return shipping     37
30 days, buyer pays return shipping                 9
Free 60 day returns                                 6
60 day returns. Buyer pays for return shipping      5
[]                                                  2
Name: Returns, dtype: int64


Nonetheless, for the purpose of this exercise I will convert it into a binary variable where no returns has a 0 value and returns of any type has a 1 value

In [323]:
# Changing available to 1 and unavailabe to 0
df['Returns'] = df['Returns'].map({'Seller does not accept returns': 0, 'Free 30 day returns': 1,
                                   '30 day returns. Buyer pays for return shipping': 1, '14 day returns. Buyer pays for return shipping': 1,
                                   '30 days, buyer pays return shipping': 1, 'Free 60 day returns': 1, '60 day returns. Buyer pays for return shipping': 1,
                                  '14 days, buyer pays return shipping': 1})

In [324]:
# Checked & Confirmed!
print(df['Returns'].value_counts())

1.0    565
0.0    558
Name: Returns, dtype: int64


<b>Shipping Currency</b>

Shipping Currency is non-binary/non-numerical; hence, we will get the count of each category level:

In [325]:
print(df['Shipping Currency'].value_counts())

$      143
C $      9
         1
Name: Shipping Currency, dtype: int64


<b>Shipping Price</b>

This is ready for final summaries (to be done later with other variables)

<b>Item Condition</b>

Item Condition is non-binary/non-numerical; hence, we will get the count of each category level:

In [326]:
print(df['Item Condition'].value_counts())

Used                        600
New                         417
Open box                     67
Seller refurbished           34
Manufacturer refurbished      7
Name: Item Condition, dtype: int64


<b>Type of AD</b>

This datatype will need to be adjusted to binary showing 1 for sponsored available and 0 for non-sponsored

In [327]:
# Checking unique values. Hence, we can apply binary factoring
df['Type of Ad'].unique()

array(['Sponsored', 'Non-Sponsored'], dtype=object)

In [328]:
# Changing sponsored to 1 and nonsponsored to 0
df['Type of Ad'] = df['Type of Ad'].map({'Sponsored': 1, 'Non-Sponsored': 0})

In [329]:
# Done. We can provided in the summaries later
df['Type of Ad'].head()

0    1
1    1
2    1
3    1
4    1
Name: Type of Ad, dtype: int64

# Time to look @ the stats! 

## Counts

In [330]:
df.groupby(['Type of Ad','Item Condition']).count()

Seller Name  Seller Score  Item Currency  \
Type of Ad Item Condition                                                       
0          Manufacturer refurbished            5             4              5   
           New                               367           320            366   
           Open box                           61            41             60   
           Seller refurbished                 30            27             30   
           Used                              536           461            536   
1          Manufacturer refurbished            2             2              2   
           New                                50            47             50   
           Open box                            6             5              6   
           Seller refurbished                  4             3              4   
           Used                               64            58             53   

                                     Item Price  Items Sold  Best Offer  \
Type of Ad Item Condition                                                 
0          Manufacturer refurbished           5           1           5   
           New                              364          85         367   
           Open box                          59           4          61   
           Seller refurbished                30           9          30   
           Used                             535          11         536   
1          Manufacturer refurbished           2           1           2   
           New                               50          20          50   
           Open box                           6           1           6   
           Seller refurbished                 4           2           4   
           Used                              53           3          64   

                                     Title  Returns  Shipping Currency  \
Type of Ad Item Condition                                                
0          Manufacturer refurbished      5        5                  0   
           New                         367      367                 48   
           Open box                     61       60                  3   
           Seller refurbished           30       30                  3   
           Used                        536      535                 78   
1          Manufacturer refurbished      2        2                  0   
           New                          50       50                  6   
           Open box                      6        6                  0   
           Seller refurbished            4        4                  1   
           Used                         64       64                 14   

                                     Shipping Price  
Type of Ad Item Condition                            
0          Manufacturer refurbished               0  
           New                                   48  
           Open box                               3  
           Seller refurbished                     3  
           Used                                  78  
1          Manufacturer refurbished               0  
           New                                    6  
           Open box                               0  
           Seller refurbished                     0  
           Used                                  14

In [331]:
df.describe()

,Seller Score,Item Price,Items Sold,Best Offer,Returns,Shipping Price,Type of Ad
count,968.000000,1108.000000,137.000000,1125.000000,1123.000000,152.000000,1125.000000
mean,99.230992,26829.938628,89.503650,0.385778,0.503117,2044.644737,0.112000
std,3.114557,8014.230896,320.207978,0.486995,0.500213,1442.341430,0.315507
min,50.000000,10000.000000,1.000000,0.000000,0.000000,199.000000,0.000000
25%,99.500000,21999.000000,2.000000,0.000000,0.000000,1000.000000,0.000000
50%,99.800000,24999.000000,6.000000,0.000000,1.000000,1749.500000,0.000000
75%,100.000000,29995.500000,22.000000,1.000000,1.000000,2500.000000,0.000000
max,100.000000,69999.000000,1896.000000,1.000000,1.000000,7000.000000,1.000000


In [332]:
print("\n")
print("\033[1mThe minimum values of each applicable attributes grouped by type of ads and item conditions can be found below: \033[0m")
print("---------------------------------------------------------------------------------------------------------------")
print(df.groupby(['Type of Ad','Item Condition']).min()[['Seller Score', 'Item Price', 'Items Sold', 'Best Offer', 'Returns', 'Shipping Price']])
print("\n")
print("\033[1mThe maximum values of each applicable attributes grouped by type of ads and item conditions can be found below: \033[0m")
print("---------------------------------------------------------------------------------------------------------------")
print(df.groupby(['Type of Ad','Item Condition']).max()[['Seller Score', 'Item Price', 'Items Sold', 'Best Offer', 'Returns', 'Shipping Price']])
print("\n")
print("\033[1mThe mean values of each applicable attributes grouped by type of ads and item conditions can be found below: \033[0m")
print("------------------------------------------------------------------------------------------------------------")
print(df.groupby(['Type of Ad','Item Condition']).mean()[['Seller Score', 'Item Price', 'Items Sold', 'Best Offer', 'Returns', 'Shipping Price']])
print("\n")
print("\033[1mThe standard deviations of each applicable attributes grouped by type of ads and item conditions can be found below: \033[0m")
print("--------------------------------------------------------------------------------------------------------------------")
print(df.groupby(['Type of Ad','Item Condition']).std()[['Seller Score', 'Item Price', 'Items Sold', 'Best Offer', 'Returns', 'Shipping Price']])

    
    



The minimum values of each applicable attributes grouped by type of ads and item conditions can be found below: 
---------------------------------------------------------------------------------------------------------------
                                     Seller Score  Item Price  Items Sold  \
Type of Ad Item Condition                                                   
0          Manufacturer refurbished          99.1     19000.0         5.0   
           New                               66.7     20000.0         1.0   
           Open box                          50.0     19000.0         1.0   
           Seller refurbished                96.6     17499.0         1.0   
           Used                              66.7     10000.0         1.0   
1          Manufacturer refurbished          99.1     19999.0         3.0   
           New                               75.0     22100.0         1.0   
           Open box                          99.3     16995.0         5.0   
   

f) How do sponsored and non-sponsored items appear to be different? Is there a variable/column that can be used to predict the sponsor/non-sponsor items?

Looking through all variables, there are certain patterns observed, some more noticable than others. I will rank them based from strongest to lowest variables that may be used for a potential prediction model. 

<b>1. Seller Score</b><br>
This is a strong indicator. Usually sponsored products are posted by users with high seller score. This was the case for all item conditions; where the minimum seller score was above 97 for all conditions except new products (75). Therefore, any user that scores < 75 is expected to post an unsponsored ad.
<br><b>2. Best Offer</b><br>
This was noticed easier with the binary variable. We can observe that products that have best offer option available are more likely to be sponsored. Especially with manufacturer refurbished which was ALWAYS providing a best offer option!
<br><b>3. Returns</b><br>
We can see from the data that on average, if products aren't offered a return option they are more likely to be un-sponsored. 
<br><b>4. Items Sold</b><br>
We noticed that in the case where there many items sold, these usually are sponsored products, except for new items.
<br><b>5. Item Price</b><br>
This is quite logical, but on average items posted for higher prices are usually those of sponsored ads, especially for items in used or manufaturer refurbished conditions.
<br><b>6. Shipping Price</b><br>
In the case that free shipping is not included, we noticed that sponsored products on average charge less for shipping.
<br><br>
<b><u>Products that have the following attributes are more likely to be sponsored:</u></b>

- Higher seller score<br>
- Best offer option available<br>
- Return option available<br>
- High number of items sold<br>
- Higher listing prices<br>
- Lower shipping prices